## Importación de librerías y declaración de funciones

In [1]:
import diseño_registro
from lector_archivo import ArchivoPresentacion

def dbl_con_ceros(num, cant):
    return "{:.2f}".format(num).replace('.','').zfill(cant)

## Apertura del TXT

In [2]:
txt = ArchivoPresentacion(r"A10_MESPASANTES_A24.txt")

## Modificación de datos dentro del TXT

### [Ejemplo] Modificación del código de liquidación para ciertos CUILs:

In [3]:
cuils_modificar = """
27257004290
20220879144
"""

# Respetar el ancho del string
codigo_anterior = '/321      '
codigo_nuevo = '/XXXxxx   '

In [ ]:
cuils_modificar = cuils_modificar.strip().split('\n')

for emp in txt.empleados:
    if emp in cuils_modificar: # Si es para todos, esta comprobación no se haría.
        for concepto in emp.conceptos:
            if concepto['codigo_concepto_liquidado'] == codigo_anterior:
                concepto['codigo_concepto_liquidado'] = codigo_nuevo
                concepto['cantidad'] = '00000'
                concepto['unidades'] = ' '

### [Ejemplo] Verificación de que la remuneracion 10 no sea menor al MNI:

In [5]:
MINIMO_NO_IMPONIBLE = 14601.14

In [6]:
for emp in txt.empleados:
    base10 = int(emp.registro4['base_imponible10'])/100.0
    
    if base10 < MINIMO_NO_IMPONIBLE:

        print(f"CUIL: {emp}")
        print("La base_imponible10 es menor al mínimo no imponible.")
        
        base8 = int(emp.registro4['base_imponible8'])/100.0
        detraccion = max(0,round(base10 - base8,2))
        
        emp.registro4['base_imponible10'] = dbl_con_ceros(MINIMO_NO_IMPONIBLE, diseño_registro.DISEÑO4['base_imponible10'])
        emp.registro4['importe_detraer'] = dbl_con_ceros(detraccion, diseño_registro.DISEÑO4['importe_detraer'])
        
        print("Se actualizaron los siguientes valores:")
        print(f" base_imponible10: {emp.registro4['base_imponible10']}")
        print(f" importe_detraer: {emp.registro4['importe_detraer']}")


CUIL: 20367518503
La base_imponible10 es menor al mínimo no imponible.
Se actualizaron los siguientes valores:
 base_imponible10: 000000001460114
 importe_detraer: 000000000000000


### [Ejemplo] Incorporación del SAC a determinados CUILs

In [7]:
# Pegar a continuación:
#  CUIL Cnt.Dias Importe
info = """
27257004290	120	100,5
20290290393	120	100,5
"""


info = info.strip().split('\n')

cuils_agregar = {}
for i in info:
    cortado = i.split('\t')
    cuils_agregar[cortado[0]] = {'dias': cortado[1], 'importe': float(cortado[2].replace(',','.'))}

cuils_agregar

{'27257004290': {'dias': '120', 'importe': 100.5},
 '20290290393': {'dias': '120', 'importe': 100.5}}

In [8]:
for emp in txt.empleados:
    if emp in list(cuils_agregar.keys()):

        dias = str(cuils_agregar[emp.cuil]['dias']).zfill(5)
        importe = dbl_con_ceros(cuils_agregar[emp.cuil]['importe'], 15)

        emp.conceptos.append({
            'registro':                     '03',
            'cuil':                         emp.cuil,
            'codigo_concepto_liquidado':    '/S13      ',
            'cantidad':                     dias,
            'unidades':                     'D',
            'importe':                      importe,
            'debcre':                       'C',
            'periodo_retro':                '      '
        })

## [Ejemplo] Asignación de seguro de vida a Pasantes

In [16]:
for emp in txt.empleados:
    if emp.registro4['modalidad_contratacion'] == '027':
        emp.registro4['marca_scvo'] = '1'

## [Ejemplo] Eliminación de información para determinados CUILs

In [3]:
cuils_eliminar = """
27431423796
20395468457
"""
cuils_eliminar = cuils_eliminar.strip().split('\n')


for emp in txt.empleados:
    if emp in cuils_eliminar:
        del emp

## Guardado del archivo modificado

In [4]:
#txt.guardar(inplace=True)
txt.guardar()

AttributeError: 'Empleado' object has no attribute 'registro2'